In [28]:
from pybbn.graph.dag import Bbn
from pybbn.graph.node import BbnNode
from pybbn.graph.variable import Variable
from pybbn.graph.edge import Edge, EdgeType
from pybbn.pptc.inferencecontroller import InferenceController
from pybbn.graph.jointree import EvidenceBuilder

In [29]:
burglary = BbnNode(Variable(0, 'Burglary', ['Yes', 'No']), [0.001, 0.999])

fire = BbnNode(Variable(1, 'Fire', ['Yes','No']), [0.002, 0.998])

In [30]:
alarm = BbnNode(Variable(2, 'Alarm', ['Yes', 'No']), [0.95, 0.05, 0.94, 0.06, 0.29, 0.71, 0.001, 0.999])

In [31]:
alice = BbnNode(Variable(3, 'Alice', ['Yes', 'No']), [0.95, 0.05, 0.05, 0.95])

bob = BbnNode(Variable(4, 'Bob', ['Yes', 'No']), [0.80, 0.20, 0.01, 0.99])

In [32]:
bbn = Bbn()\
    .add_node(burglary)\
    .add_node(fire)\
    .add_node(alarm)\
    .add_node(alice)\
    .add_node(bob)\
    .add_edge(Edge(burglary, alarm, EdgeType.DIRECTED))\
    .add_edge(Edge(fire,alarm, EdgeType.DIRECTED))\
    .add_edge(Edge(alarm, alice, EdgeType.DIRECTED))\
    .add_edge(Edge(alarm, bob, EdgeType.DIRECTED))

In [33]:
join_tree = InferenceController.apply(bbn)
for node, posteriors in join_tree.get_posteriors().items():
    p = ', '.join([f'{val}={prob:.5f}' for val, prob in posteriors.items()])
    print(f'{node} : {p}')

Alarm : Yes=0.00252, No=0.99748
Alice : Yes=0.05226, No=0.94774
Bob : Yes=0.01199, No=0.98801
Fire : Yes=0.00200, No=0.99800
Burglary : Yes=0.00100, No=0.99900


In [34]:
orignal = join_tree.__copy__()

In [35]:
# Define a function for printing marginal probabilities
def print_probs():
    for node in join_tree.get_bbn_nodes():
        potential = join_tree.get_bbn_potential(node)
        print("Node:", node)
        print("Values:")
        print(potential)
        print('----------------')

# Use the above function to print marginal probabilities
print_probs()

Node: 2|Alarm|Yes,No
Values:
2=Yes|0.00252
2=No|0.99748
----------------
Node: 3|Alice|Yes,No
Values:
3=Yes|0.05226
3=No|0.94774
----------------
Node: 4|Bob|Yes,No
Values:
4=Yes|0.01199
4=No|0.98801
----------------
Node: 1|Fire|Yes,No
Values:
1=Yes|0.00200
1=No|0.99800
----------------
Node: 0|Burglary|Yes,No
Values:
0=Yes|0.00100
0=No|0.99900
----------------


In [36]:
# To add evidence of events that happened so probability distribution can be recalculated
def evidence(ev, nod, cat, val):
    ev = EvidenceBuilder().with_node(join_tree.get_bbn_node_by_name(nod)).with_evidence(cat, val).build()
    join_tree.set_observation(ev)

p(alarm) = p(alarm| burglary, fire) * p(burglary) * p(fire) + p(alarm| burglary, ~fire) * p(burglary) * p(~fire) + p(alarm| ~burglary, fire) * p(~burglary) * p(fire) + p(alarm| ~burglary, ~fire) * p(~fire) * p(~fire)


p(alice) = p(alice| alarm) * p(alarm) + p(alice| ~alarm) * p(~alarm)

In [37]:
val = join_tree.get_bbn_potential(alarm).entries[0].value

In [44]:
p_burglary_yes = join_tree.get_bbn_potential(burglary).entries[0].value
p_burglary_no = join_tree.get_bbn_potential(burglary).entries[1].value

p_fire_yes = join_tree.get_bbn_potential(fire).entries[0].value
p_fire_no = join_tree.get_bbn_potential(fire).entries[1].value

In [45]:
# p(alarm| burglary, fire)
evidence('ev1', 'Burglary', 'Yes', 1.0)
evidence('ev2', 'Fire', 'Yes', 1.0)
p_alarm_yes_burglary_yes_fire_yes = join_tree.get_bbn_potential(alarm).entries[0].value
join_tree.unobserve([burglary, fire])

# p(alarm| burglary, ~fire)
evidence('ev1', 'Burglary', 'Yes', 1.0)
evidence('ev2', 'Fire', 'No', 1.0)
p_alarm_yes_burglary_yes_fire_no = join_tree.get_bbn_potential(alarm).entries[0].value
join_tree.unobserve([burglary, fire])

# p(alarm| ~burglary, fire)
evidence('ev1', 'Burglary', 'No', 1.0)
evidence('ev2', 'Fire', 'Yes', 1.0)
p_alarm_yes_burglary_no_fire_yes = join_tree.get_bbn_potential(alarm).entries[0].value
join_tree.unobserve([burglary, fire])

# p(alarm| ~burglary, ~fire)
evidence('ev1', 'Burglary', 'No', 1.0)
evidence('ev2', 'Fire', 'No', 1.0)
p_alarm_yes_burglary_no_fire_no = join_tree.get_bbn_potential(alarm).entries[0].value
join_tree.unobserve([burglary, fire])

In [46]:
p_alarm_yes = p_alarm_yes_burglary_yes_fire_yes * p_burglary_yes * p_fire_yes + p_alarm_yes_burglary_yes_fire_no * p_burglary_yes * p_fire_no + p_alarm_yes_burglary_no_fire_yes * p_burglary_no * p_fire_yes + p_alarm_yes_burglary_no_fire_no * p_burglary_no * p_fire_no

In [47]:
print(p_alarm_yes_burglary_yes_fire_yes, p_burglary_yes, p_fire_yes)

0.9500000000000001 0.001 0.002


In [48]:
p_alarm_yes

0.002516442

In [49]:
p_alarm_no = 1 - p_alarm_yes

In [50]:
p_alarm_no

0.997483558

In [51]:
#p(alice| alarm)
evidence('ev1', 'Alarm', 'Yes', 1.0)
p_alice_yes_alarm_yes = join_tree.get_bbn_potential(alice).entries[0].value
join_tree.unobserve([alarm])

#p(alice| ~alarm)
evidence('ev1', 'Alarm', 'No', 1.0)
p_alice_yes_alarm_no = join_tree.get_bbn_potential(alice).entries[0].value
join_tree.unobserve([alarm])

In [52]:
p_alice_yes = p_alice_yes_alarm_yes * p_alarm_yes + p_alice_yes_alarm_no * p_alarm_no

In [53]:
p_alice_yes

0.052264797800000005

In [54]:
p_alice_no = 1 - p_alice_yes

In [55]:
p_alarm_no

0.997483558